In [107]:
import numpy as np
import pandas as pd
import scipy
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
#from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import plotly.express as px
import time 



## The pendulum


<img src="Figs/pendulum.svg" alt="drawing" width="200"/>

Equation of motion for the pendulum reads: 

$$
\ddot{\theta}+\omega^2\sin{\theta}=0
$$
where $\omega =\sqrt{\frac{g}{L}}.
### The linear pendulum
In introductory course in physics we often use the linearized version. Assuming that $\theta \ll 1$ gives

$$
sin(\theta) \sim x-\frac{1}{3} x^3 + O(x^5)
$$

and we end up with the linear pendulum

$$
\ddot{\theta}+\omega^2\theta=0
$$

with solution
$$
\theta= \Theta \sin(\omega t+\varphi)
$$

where the constants $\Theta$ and $\varphi$ are determined by the initial conditions. Assuming that $\theta(0)=\theta_0$ and $\dot{\theta}(0)=\dot{\theta}_0$ we have 
$$
\begin{array}{c}
 \Theta \sin(\varphi) = \theta_0 \\ \\
 \omega \Theta  \cos(\varphi)=\dot{\theta_0}
\end{array}
$$




## Define the equations


To be able to solve the second order ordinary differential equations (ODE), the easiest will be to make them a system of first order equation. A general second order ODE 
$$
 \ddot{x} = F(x,\dot{x})
$$
can be made into a system: 
$$
\begin{array}{l}
 \dot{x}=y \\
 \dot{y}=F(x,y)
 \end{array}
$$

Defining the right hand sides of the equations for the pendulum

In [115]:
def lin_pendulum(t, x, L, g):
    dx0=x[1]
    dx1= - g/L*x[0]
    return [dx0, dx1]

def nonlin_pendulum(t, x, L, g):
    dx0=x[1]
    dx1= - g/L*np.sin(x[0])
    return [dx0, dx1]

def lin_solution(t,x0= [-np.pi*0.25, 0],L=2,g=9.81):
    omega=np.sqrt(g/L)
    A=x0[0]
    B=x0[1]/omega
    theta= A*np.cos(omega*t)+B*np.sin(omega*t)
    theta_dot= - A*omega*np.sin(omega*t)+B*omega*np.cos(omega*t)
    x=L*np.sin(theta)
    y=-L*np.cos(theta)
    
    data={'t':t}
          
    #, 'theta': theta.T,'theta_dot':theta_dot.T , 'x': x.T, 'y':y.T}
    out=pd.DataFrame(data=data,index=list(range(len(data))))
    out['theta']=theta
    out['theta_dot']=theta_dot
    out['x']=x
    out['y']=y
    out['label']='Analytic linear'
    return out

def a_solution(func=lin_pendulum,x0 = [-np.pi*0.25, 0],t_max=10,dt=0.1, L=2,):
    g   =9.81
    tspan = (0, t_max)
    t=np.arange(tspan[0], tspan[1], dt)
    sol=solve_ivp(lambda t, x: func(t, x, L, g), tspan, x0, t_eval=t)

    columns=['t','theta','theta_dt']
    y1, y2 = sol.y


    df = pd.DataFrame({'t':sol.t})
    #df["t"] = sol.t
    df['theta']=y1
    df['theta_dot']=y2
    df['x']=L*np.sin(df.theta)
    df['y']=-L*np.cos(df.theta)
    df['label']=func.__name__
    return df

In [116]:


L = 2
t0=time.perf_counter()
df=a_solution(func=lin_pendulum)
dt=time.perf_counter()-t0
print('Elapsed time (linear):',dt)

t0=time.perf_counter()
ndf=a_solution(func=nonlin_pendulum)
dt=time.perf_counter()-t0
print('Elapsed time (non-linear):',dt)

t0=time.perf_counter()
adf=lin_solution(t=np.arange(tspan[0], tspan[1], dt))
dt=time.perf_counter()-t0
print('Elapsed time (analytic):',dt)

ddf=pd.concat([df,ndf,adf])


Elapsed time (linear): 0.005484374996740371
Elapsed time (non-linear): 0.003593833011109382


ValueError: Length of values (2783) does not match length of index (1)

## Timing



In [117]:
t0=time.perf_counter()
lin_solution(t=1)
dt=time.perf_counter()-t0
print('Elapsed time (analytic):',dt)

Elapsed time (analytic): 0.001746790949255228


In [97]:
px.scatter(ddf, x="x", y="y", animation_frame="t",  color='label', size_max=55, range_x=[-2,2], range_y=[-2.1,0])


In [102]:
px.line(ddf,x='t',y='theta',color='label')



In [130]:
for name, group in ddf.groupby('label'):
    res=ddf[]

Analytic linear
lin_pendulum
nonlin_pendulum


In [139]:
ddf['theta'].where(ddf['label']=='lin_pendulum').dropna()-ddf['theta'].where(ddf['label']=='Analytic linear').dropna()

0       0.000000
1       0.019157
2       0.075710
3       0.166903
4       0.288215
          ...   
3113         NaN
3114         NaN
3115         NaN
3116         NaN
3117         NaN
Name: theta, Length: 3118, dtype: float64